## Model

In [1]:
from prismatic.models import get_llm_backbone_and_tokenizer, get_vision_backbone_and_transform, get_vlm

/home/localstorage/miniconda3/envs/prismatic/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
vision_backbone, image_transform = get_vision_backbone_and_transform(
        "dinosiglip-vit-so-384px", image_resize_strategy="resize-naive"
    )

04/13 [10:16:51] INFO     | >> Loading pretrained weights from Hugging Face hub                     ]8;id=318321;file:///home/localstorage/miniconda3/envs/prismatic/lib/python3.10/site-packages/timm/models/_builder.py\_builder.py]8;;\:]8;id=809947;file:///home/localstorage/miniconda3/envs/prismatic/lib/python3.10/site-packages/timm/models/_builder.py#186\186]8;;\
                          (timm/vit_large_patch14_reg4_dinov2.lvd142m)                                             

04/13 [10:16:52] INFO     | >>  Safe alternative available for 'pytorch_model.bin' (as                  ]8;id=431095;file:///home/localstorage/miniconda3/envs/prismatic/lib/python3.10/site-packages/timm/models/_hub.py\_hub.py]8;;\:]8;id=647772;file:///home/localstorage/miniconda3/envs/prismatic/lib/python3.10/site-packages/timm/models/_hub.py#180\180]8;;\
                          'model.safetensors'). Loading weights using safetensors.                                 

                 INFO     | >> Resized position embedding: (37, 37) to (27, 27).                    ]8;id=284552;file:///home/localstorage/miniconda3/envs/prismatic/lib/python3.10/site-packages/timm/layers/pos_embed.py\pos_embed.py]8;;\:]8;id=467854;file:///home/localstorage/miniconda3/envs/prismatic/lib/python3.10/site-packages/timm/layers/pos_embed.py#55\55]8;;\

04/13 [10:17:00] INFO     | >> Loading pretrained weights from Hugging Face hub                     ]8;id=374566;file:///home/localstorage/miniconda3/envs/prismatic/lib/python3.10/site-packages/timm/models/_builder.py\_builder.py]8;;\:]8;id=430082;file:///home/localstorage/miniconda3/envs/prismatic/lib/python3.10/site-packages/timm/models/_builder.py#186\186]8;;\
                          (('timm/ViT-SO400M-14-SigLIP-384', 'open_clip_pytorch_model.bin'))                       

                 INFO     | >>  Safe alternative available for 'open_clip_pytorch_model.bin' (as        ]8;id=288606;file:///home/localstorage/miniconda3/envs/prismatic/lib/python3.10/site-packages/timm/models/_hub.py\_hub.py]8;;\:]8;id=49785;file:///home/localstorage/miniconda3/envs/prismatic/lib/python3.10/site-packages/timm/models/_hub.py#180\180]8;;\
                          'open_clip_model.safetensors'). Loading weights using safetensors.                       

In [3]:
llm_backbone, tokenizer = get_llm_backbone_and_tokenizer(
        "phi2_base", llm_max_length=2048
    )

04/13 [10:17:22] INFO     | >>     |=> Loading phi2 LLM from `microsoft/phi-2`                      ]8;id=503762;file:///home/DSO_SSD/Nithish/LVLMs/prismatic-vlms/prismatic/models/backbones/llm/base_llm.py\base_llm.py]8;;\:]8;id=980734;file:///home/DSO_SSD/Nithish/LVLMs/prismatic-vlms/prismatic/models/backbones/llm/base_llm.py#118\118]8;;\

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.28s/it]


04/13 [10:17:26] INFO     | >>     |=> Loading phi2 (Fast) Tokenizer via the AutoTokenizer API      ]8;id=268065;file:///home/DSO_SSD/Nithish/LVLMs/prismatic-vlms/prismatic/models/backbones/llm/base_llm.py\base_llm.py]8;;\:]8;id=824709;file:///home/DSO_SSD/Nithish/LVLMs/prismatic-vlms/prismatic/models/backbones/llm/base_llm.py#163\163]8;;\

In [4]:
vlm = get_vlm(
    model_id = "dino-siglip-phi2",
    arch_specifier = "gelu-mlp",
    vision_backbone = vision_backbone,
    llm_backbone = llm_backbone,
)
print(vlm.llm_backbone)

Phi2LLMBackbone(
  (llm): PeftModelForCausalLM(
    (base_model): LoraModel(
      (model): PhiForCausalLM(
        (model): PhiModel(
          (embed_tokens): Embedding(50295, 2560)
          (embed_dropout): Dropout(p=0.0, inplace=False)
          (layers): ModuleList(
            (0-31): 32 x PhiDecoderLayer(
              (self_attn): PhiAttention(
                (q_proj): lora.Linear(
                  (base_layer): Linear(in_features=2560, out_features=2560, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=2560, out_features=8, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=8, out_features=2560, bias=False)
                  )
                  (lora_embedding_A): ParameterDict()
                  (lora_embedding_B): ParameterD

In [5]:
vlm.freeze_backbones(stage="finetune")

print("Total params = ",sum(p.numel() for p in vlm.parameters()))
print("Trainable params = ",sum(p.numel() for p in vlm.parameters() if p.requires_grad))

#       original
# Total params =  3,519,119,543
# Trainable params =  3,519,119,543

#       align
# Total params =  3,519,119,543
# Trainable params =  12,129,280

# "finetune" without LoRA ; commented the line # self.llm_backbone.requires_grad_(True) in prismatic.py
# Total params =  3519119543
# Trainable params =  2787178615

# "finetune" with LoRA ; commented the line # self.llm_backbone.requires_grad_(True) in prismatic.py
# Total params =  3521740983
# Trainable params =  14750720

04/13 [10:17:48] INFO     | >>     |=> [Frozen]    🥶 =>> Vision Backbone                          ]8;id=264245;file:///home/DSO_SSD/Nithish/LVLMs/prismatic-vlms/prismatic/models/vlms/prismatic.py\prismatic.py]8;;\:]8;id=899004;file:///home/DSO_SSD/Nithish/LVLMs/prismatic-vlms/prismatic/models/vlms/prismatic.py#170\170]8;;\
                          `dinosiglip-vit-so-384px`                                                                

                 INFO     | >>     |=> [TRAINABLE] 🔥 =>> LLM Backbone `phi2_base`                 ]8;id=15582;file:///home/DSO_SSD/Nithish/LVLMs/prismatic-vlms/prismatic/models/vlms/prismatic.py\prismatic.py]8;;\:]8;id=120664;file:///home/DSO_SSD/Nithish/LVLMs/prismatic-vlms/prismatic/models/vlms/prismatic.py#171\171]8;;\

                 INFO     | >>     |=> [TRAINABLE] 🔥 =>> Projector `gelu-mlp`                     ]8;id=193329;file:///home/DSO_SSD/Nithish/LVLMs/prismatic-vlms/prismatic/models/vlms/prismatic.py\prismatic.py]8;;\:]8;id=19894;file:///home/DSO_SSD/Nithish/LVLMs/prismatic-vlms/prismatic/models/vlms/prismatic.py#172\172]8;;\

Total params =  3521740983
Trainable params =  14750720


In [12]:
prompter = vlm.get_prompt_builder()
prompter = PurePromptBuilder(model_family="phi2")
print(prompter.model_family, prompter.bos, prompter.eos, prompter.prompt, prompter.turn_count)

phi2 <|endoftext|> <|endoftext|>  0


In [14]:
prompter.add_turn(
    role = "human",
    message = "<image> How many apples are in this picture?"
)
print("\n",prompter.prompt, prompter.turn_count)

prompter.add_turn(
    role = "gpt",
    message = "There are 10 apples."
)
print("\n",prompter.prompt, prompter.turn_count)

prompter.add_turn(
    role = "human",
    message = "<image> How many oranges are in this picture?"
)
print("\n",prompter.prompt, prompter.turn_count)

prompter.add_turn(
    role = "gpt",
    message = "There are no oranges in this picture."
)
print("\n",prompter.prompt, prompter.turn_count)

# PurePromptBuilder doesnot include system prompt yet!
# <|endoftext|> appearing in chat history ???


 In: How many apples are in this picture?
Out:  1

 In: How many apples are in this picture?
Out: There are 10 apples.<|endoftext|> 2

 In: How many apples are in this picture?
Out: There are 10 apples.<|endoftext|>In: How many oranges are in this picture?
Out:  3

 In: How many apples are in this picture?
Out: There are 10 apples.<|endoftext|>In: How many oranges are in this picture?
Out:  <|endoftext|> 4
